In [12]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [13]:
df = pd.read_csv('./housing_data.csv', parse_dates=['sold_date'])
df = df.drop_duplicates()
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14325 entries, 0 to 15076
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   address       14325 non-null  object        
 1   housing_type  14325 non-null  object        
 2   sold_date     14325 non-null  datetime64[ns]
 3   n_beds        14325 non-null  object        
 4   n_bath        14325 non-null  object        
 5   n_garage      14325 non-null  object        
 6   prices        14325 non-null  int64         
 7   suburb        14325 non-null  object        
 8   state         14325 non-null  object        
 9   postcode      14325 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 1.2+ MB


In [14]:
def fix_state(df):
    df1 = df.loc[df.state != 'VIC'].copy()
    df1['suburb'] = df1['suburb'] + ' ' + df['state']
    df1 = df1.drop('state', axis=1)
    df1.postcode = df1.postcode.str.strip()
    df1[['state', 'postcode']] = df1.postcode.str.split(" ", expand=True)
    return df1

df1 = fix_state(df)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6614 entries, 4295 to 11611
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   address       6614 non-null   object        
 1   housing_type  6614 non-null   object        
 2   sold_date     6614 non-null   datetime64[ns]
 3   n_beds        6614 non-null   object        
 4   n_bath        6614 non-null   object        
 5   n_garage      6614 non-null   object        
 6   prices        6614 non-null   int64         
 7   suburb        6614 non-null   object        
 8   postcode      6614 non-null   object        
 9   state         6614 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 826.4+ KB


In [15]:
df_final = pd.concat([df , df1])
df_final = df_final.drop_duplicates(subset='address', keep='last')

In [16]:
df_final['country'] = 'AUSTRALIA'

In [17]:
df_final['whole_address'] = df_final[['address', 'suburb', 'state', 'postcode', 'country']].T.agg(', '.join)
df_final

,address,housing_type,sold_date,n_beds,n_bath,n_garage,prices,suburb,state,postcode,country,whole_address
1,211/9 Morton Avenue,Apartment,2022-08-09,2,1,1,470000,CARNEGIE,VIC,3163,AUSTRALIA,"211/9 Morton Avenue, CARNEGIE, VIC, 3163, AUST..."
2,203/3 Morton Avenue,Apartment,2022-08-03,2,1,1,441000,CARNEGIE,VIC,3163,AUSTRALIA,"203/3 Morton Avenue, CARNEGIE, VIC, 3163, AUST..."
3,5/87 Coorigil Road,Apartment,2022-07-28,1,1,1,340000,CARNEGIE,VIC,3163,AUSTRALIA,"5/87 Coorigil Road, CARNEGIE, VIC, 3163, AUSTR..."
4,2/82-84 Coorigil Road,Apartment,2022-07-28,2,1,1,495050,CARNEGIE,VIC,3163,AUSTRALIA,"2/82-84 Coorigil Road, CARNEGIE, VIC, 3163, AU..."
5,203/284 Neerim Road,Apartment,2022-07-05,2,1,1,417000,CARNEGIE,VIC,3163,AUSTRALIA,"203/284 Neerim Road, CARNEGIE, VIC, 3163, AUST..."
...,...,...,...,...,...,...,...,...,...,...,...,...
11607,466 Kooyong Road,House,2011-02-21,4,2,2,1655000,CAULFIELD SOUTH,VIC,3162,AUSTRALIA,"466 Kooyong Road, CAULFIELD SOUTH, VIC, 3162, ..."
11608,1/16 Keeron Street,Townhouse,2010-12-20,3,2,2,911000,CAULFIELD SOUTH,VIC,3162,AUSTRALIA,"1/16 Keeron Street, CAULFIELD SOUTH, VIC, 3162..."
11609,4 Leopold Street,House,2010-12-20,3,2,3,1095000,CAULFIELD SOUTH,VIC,3162,AUSTRALIA,"4 Leopold Street, CAULFIELD SOUTH, VIC, 3162, ..."
11610,2 Olinda Street,House,2010-12-13,2,1,1,655000,CAULFIELD SOUTH,VIC,3162,AUSTRALIA,"2 Olinda Street, CAULFIELD SOUTH, VIC, 3162, A..."


In [18]:
geolocator = Nominatim(user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36')

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df_final['location'] = df_final['whole_address'].apply(geocode)
df_final

# df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)


KeyboardInterrupt: 